In [64]:
#Imports

### Unfortunately, UMAP takes a while to import. One of its dependencies (pynndescent) uses numba, 
### which is the cause of the performance bottleneck here.

import cv2
import hdbscan
import json 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import re
import seaborn as sns
import skimage 
import umap
import umap.plot

from bokeh.embed import json_item
from bokeh.plotting import show as bokeh_show, output_notebook
from glob import glob
from hdbscan import HDBSCAN
from IPython.display import display, HTML, Image, Javascript
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [65]:
#Constants
IMG_SIZE = 256
IMAGES_DIR = '../images/final_pigmentation_catalogue_2016'
TARGET_IMAGES_DIR = '../images/final_to_match'
MAX_FILES = 300

We first load the labelled images.

In [66]:
image_dirs = Path(IMAGES_DIR) 

labels = []
image_files = []

for image_dir in image_dirs.glob('*'):
    label = image_dir.stem
    for file in image_dir.glob('*'):
        ext = file.suffix.lower()
        if ext != ".png": continue
        image_files.append(file)
        labels.append(label)
    if len(image_files) >= MAX_FILES: break

Then we apply additional pre-processing.

In [67]:
images = []
names = []
files = []

for i, file in enumerate(image_files):
    file_str = str(file)
    image = cv2.imread(file_str)
    names.append(file.stem)
    files.append(file_str)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image.astype('float32')
#     #image = (image - image.mean(axis=(0, 1), keepdims = True)) / image.std(axis=(0, 1), keepdims = True)
#     #image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)
    images.append(image)
    
    progress = i/MAX_FILES * 100
    if progress % 5 == 0: print(f'{progress}% done')
        
print("Complete!")

0.0% done
5.0% done
10.0% done
15.0% done
20.0% done
25.0% done
30.0% done
35.0% done
40.0% done
45.0% done
50.0% done
60.0% done
65.0% done
70.0% done
75.0% done
80.0% done
85.0% done
90.0% done
95.0% done
100.0% done
Complete!


The image data is converted into a numpy array.

In [68]:
images = np.asarray(images)
images.shape

(312, 256, 256)

Then flattened into 2D.

In [69]:
data = images.reshape((images.shape[0], -1))
#data = StandardScaler().fit_transform(data)
data.shape

(312, 65536)

UMAP will be fitted to the data. This time, the class labels will be provided.

A collection of all the class labels:

In [70]:
y_encoder = LabelEncoder()
y_encoder.fit(labels)
y_target = y_encoder.transform(labels)
set(labels)

{'0548',
 '0550',
 '0600',
 '0619',
 '0627',
 '0659',
 '0660',
 '0664',
 '0685',
 '0693',
 '0698',
 '0704',
 '0710',
 '0718',
 '0722',
 '0723',
 '0727'}

In [72]:
mapper = umap.UMAP(
    n_neighbors = 200,
    min_dist = 0.5,
    n_components = 2,
    metric = 'euclidean',
    random_state = 100,
    densmap = False
).fit(data, y = y_target)

umap_res = mapper.transform(data)

We apply HDBSCAN to the UMAP results.

In [73]:
cluster = HDBSCAN(
    algorithm ='best', 
    approx_min_span_tree = True,
    gen_min_span_tree = False, 
    leaf_size = 40, 
    metric='euclidean', 
    min_cluster_size = 10,
    min_samples = 10, 
    p = None
).fit(umap_res)

In [74]:
hover_data = pd.DataFrame(
    index = np.arange(
        data.shape[0] 
    ), 
    data = {
        'index' : np.arange(data.shape[0]),
        'name': names,
        "file": files,
        "identity": labels,
        'cluster_class': cluster.labels_,
        'probability': cluster.probabilities_,
        'outlier': [ 1 if item == -1 else 0 for item in cluster.labels_ ]
    }
)

In [75]:
hover_data.head(10).drop("index", axis = 1)

,name,file,identity,cluster_class,probability,outlier
0,HG_110923_100_E2_N3,../images/final_pigmentation_catalogue_2016/05...,0548,4,0.801201,0
1,HG_130711_093_E4_CL_N7,../images/final_pigmentation_catalogue_2016/05...,0548,4,0.860212,0
2,HG_130720_0333_E1_CL_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,0.884215,0
3,HG_130720_0334_E1_CL_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,0.710521,0
4,HG_130720_0359_E1_CL_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,0.908024,0
5,HG_130720_0360_E1_CL_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,1.000000,0
6,HG_130720_0361_E1_CL_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,1.000000,0
7,HG_130720_0362_E1_CL_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,1.000000,0
8,HG_130720_0363_E1_CL_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,0.985560,0
9,HG_130720_0462_E1_KR_AII,../images/final_pigmentation_catalogue_2016/05...,0548,4,0.542459,0


In [76]:
p = umap.plot.interactive(mapper, labels = labels,hover_data = hover_data, point_size = 5)
output_notebook()
bokeh_show(p)

Loading BokehJS ...

The front-end has been built using Vue.js. Therefore, the data must be consumable using a JS-friendly format. We can convert the Bokeh plot into a JSON blob, and visualise it using Bokeh.js. 

In [77]:
p_json = json.dumps(json_item(p))
p_json[0:150]

'{"target_id": null, "root_id": "1731", "doc": {"defs": [], "roots": {"references": [{"attributes": {"background_fill_color": "white", "below": [{"id":'

In [78]:
display(HTML('<div id="umap"></div>'))
Javascript(f'''Bokeh.embed.embed_item({p_json}, "umap")''')

<IPython.core.display.Javascript object>

Now, we want to apply UMAP to the unlabelled data.

In [79]:
sup_files = Path(TARGET_IMAGES_DIR) 
sup_image_files = []

for file in sup_files.rglob('*'):
    ext = file.suffix.lower()
    if ext == ".jpg": sup_image_files.append(file)
    if len(sup_image_files) >= MAX_FILES: break
    
sup_image_files[0:4]

[PosixPath('../images/final_to_match/HG_170305_0027_DE_P4.jpg'),
 PosixPath('../images/final_to_match/HG_170305_0041_GF_P5.JPG'),
 PosixPath('../images/final_to_match/HG_170305_0046_GF_P6.JPG'),
 PosixPath('../images/final_to_match/HG_170305_0051_GF_P1.JPG')]

In [80]:
sup_images = []
sup_names = []
sup_files = []

for i, file in enumerate(sup_image_files):
    file_str = str(file)
    image = cv2.imread(file_str)
    sup_names.append(file.stem)
    sup_files.append(file_str)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image.astype('float32')
    #image = (image - image.mean(axis=(0, 1), keepdims = True)) / image.std(axis=(0, 1), keepdims = True)
    #image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)
    sup_images.append(image)
    
    progress = i/MAX_FILES * 100
    if progress % 5 == 0: print(f'{progress}% done')
        
print("Complete!")

0.0% done
5.0% done
10.0% done
15.0% done
20.0% done
25.0% done
30.0% done
35.0% done
40.0% done
45.0% done
50.0% done
60.0% done
65.0% done
70.0% done
75.0% done
80.0% done
85.0% done
90.0% done
95.0% done
Complete!


In [81]:
sup_images = np.asarray(sup_images)
sup_images.shape

(300, 256, 256)

In [82]:
sup_data = sup_images.reshape((sup_images.shape[0], -1))
#data = StandardScaler().fit_transform(data)
sup_data.shape

(300, 65536)

In [83]:
sup_umap_res = mapper.transform(sup_data)

In [84]:
sup_cluster = HDBSCAN(
    algorithm ='best', 
    approx_min_span_tree = True,
    gen_min_span_tree = False, 
    leaf_size = 40, 
    metric='euclidean', 
    min_cluster_size = 15,
    min_samples = 15, 
    p = None
).fit(sup_umap_res)

In [85]:
sup_umap_res.shape

(300, 2)

In [86]:
sup_hover_data = pd.DataFrame(
    index = np.arange(
        sup_data.shape[0] 
    ), 
    data = {
        'index' : np.arange(sup_data.shape[0]),
        'name': sup_names,
        "file": sup_files,
        'class': sup_cluster.labels_,
        'probability': sup_cluster.probabilities_,
        'outlier': [ 1 if item == -1 else 0 for item in sup_cluster.labels_ ]
    }
)

In [87]:
sup_hover_data.head(10).drop("index", axis = 1)

,name,file,class,probability,outlier
0,HG_170305_0027_DE_P4,../images/final_to_match/HG_170305_0027_DE_P4.jpg,0,0.546100,0
1,HG_170305_0041_GF_P5,../images/final_to_match/HG_170305_0041_GF_P5.JPG,0,0.937345,0
2,HG_170305_0046_GF_P6,../images/final_to_match/HG_170305_0046_GF_P6.JPG,0,0.769964,0
3,HG_170305_0051_GF_P1,../images/final_to_match/HG_170305_0051_GF_P1.JPG,0,0.586742,0
4,HG_170305_0067_GF_P7,../images/final_to_match/HG_170305_0067_GF_P7.JPG,2,1.000000,0
5,HG_170305_0073_GF_N3,../images/final_to_match/HG_170305_0073_GF_N3.JPG,1,1.000000,0
6,HG_170305_0091_GF_P8,../images/final_to_match/HG_170305_0091_GF_P8.JPG,0,1.000000,0
7,HG_170305_0110_GF_N6,../images/final_to_match/HG_170305_0110_GF_N6.JPG,0,0.892307,0
8,HG_170305_0115_GF_N5,../images/final_to_match/HG_170305_0115_GF_N5.JPG,1,1.000000,0
9,HG_170305_0140_GF_N4,../images/final_to_match/HG_170305_0140_GF_N4.JPG,0,0.525718,0


In [88]:
sup_mapper = umap.UMAP()
sup_mapper.embedding = sup_umap_res
p = umap.plot.interactive(sup_mapper, labels = sup_cluster.labels_,hover_data = sup_hover_data, point_size = 5)
output_notebook()
bokeh_show(p)

Loading BokehJS ...